In [ ]:
import requests
import json
import os
import time
outputpath="/content/test.json"
f = open("/content/quotestable.json",encoding='utf-8')

# returns JSON object as
# a dictionary
data1 = json.load(f)
# os.remove(outputpath)
f = open(outputpath, "a",encoding="utf-8")
idcount=0
counter=0
list1=[]
#clean the dataset of duplicate authors
authorset=set()
for count,i in enumerate(data1):
  authorset.add(i["author"].strip())
print(len(authorset))
for auth in authorset:
  url = 'https://query.wikidata.org/sparql'
  query = '''
  SELECT ?item ?prefLabel ?itemDesc ?image ?birthLocation ?birthLocationLabel ?occupation ?occupationLabel ?coord ?region ?regionLabel WHERE {
        VALUES ?prefLabel {  "'''+auth+'''"@en }
        ?item rdfs:label|skos:altLabel ?prefLabel;
             wdt:P31 wd:Q5;
             wdt:P19 ?birthLocation.
             OPTIONAL {?item wdt:P18 ?image.}
             OPTIONAL {?item schema:description ?itemDesc.
             FILTER ( lang(?itemDesc) = "en" )}
             OPTIONAL {?birthLocation wdt:P625 ?coord. }
             OPTIONAL {?birthLocation wdt:P17 ?region. }
             OPTIONAL { ?item wdt:P106 ?occupation . }
             
SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
  '''
  time.sleep(1)
  try:
    r = requests.get(url, params = {'format': 'json', 'query': query}, timeout=5)
    data = r.json()
  except Exception as e:
    print(e)
    exit(1)
  try:
    a =data["results"]['bindings'][-1]["birthLocationLabel"]["value"]
    
  except:
    continue


  authordict={}
  authordict["name"] = auth
  authordict["birthlocation"] = a
  if "image" in data["results"]['bindings'][-1]:
    authordict["image"] =  data["results"]['bindings'][-1]["image"]["value"]
  else:
    authordict["image"] = None
  if "itemDesc" in data["results"]['bindings'][-1]:
    authordict["description"] =  data["results"]['bindings'][-1]["itemDesc"]["value"]
  else:
    authordict["description"] = None
  if "coord" in data["results"]['bindings'][-1]:
    authordict["coords"] =  data["results"]['bindings'][-1]["coord"]["value"]
  else:
    authordict["coords"] = None
  if "regionLabel" in data["results"]['bindings'][-1]:
    authordict["regionLabel"] =  data["results"]['bindings'][-1]["regionLabel"]["value"]
  else:
    authordict["regionLabel"] = None
  jobs=[]
  for x in data["results"]['bindings']:
    if "occupationLabel" in x:
      jobs.append(x["occupationLabel"]["value"])
  authordict["occupation"] = jobs
  print(str(authordict) + ",")
  list1.append(authordict)
with open("authors.json", 'w') as f2:
    json.dump(list1, f2)
print(list1)